# BAG3D BGT Intersection
This notebook computes the optimial intersection height between 3DBAG building and 3Dfied BGT buildings based on point clouds.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import trimesh
import numpy as np
import matplotlib.pyplot as plt

# Suppress loading warnings
import logging
logger = logging.getLogger("trimesh")
logger.setLevel(logging.ERROR)

from src.scrapers.bag_scraper import get_bag_by_tile_codes
from src.scrapers.bag3d_scraper import get_bag3d_as_json
from src.scrapers.bgt_scraper import get_bgt_by_tile_codes

from src.intersect import intersect

### Load all city data-sources

In [ ]:
# Load necesarry data sources
tile_codes = ['2445_9723', '2445_9724', '2445_9725', '2446_9723', '2446_9724', '2446_9725', '2447_9724', '2447_9725', '2447_9726', '2448_9724', '2448_9725', '2448_9726', '2449_9724', '2449_9725', '2449_9726', '2450_9724', '2450_9725', '2450_9726',]

# 2.5D building model (Can take a minute)
city_model = get_bag3d_as_json(tile_codes)

# 2D city footprints
city_map = get_bgt_by_tile_codes(tile_codes, padding=5)

# 2D building outlines
city_outline = get_bag_by_tile_codes(tile_codes, padding=5)

## Preprocess point clouds

In [ ]:
# TODO

## Compute optimal intersection height
And save the results as `.obj`

### Parameter explanation
* `idx`: List of strings containing building idx (bag_ids) for the dataloader.
* `out_folder`: Output folder.
* `dataset='Amsterdam'`: Switch for different datasets.
* `stepsize=0.1`: Defines the interval to check the intersection. Defaults to 0.1 meters.
* `N=10000`: Number of evaluation samples. Higher is more accurate but slower.
* `improvement_threshold=0.0`: Only use the new building model when the improvement is higher then this threshold.
* `bottom_buffer=2.5`: Only allow intersections above this buffer with respect to the 3dbag bottom.
* `top_buffer=2.5`: Only allow intersections bellow this buffer with respect to the 3dbag top.
* `smooth=True`: Add smoothing to reduce the impact of outliers.

In [ ]:
# List of strings containing building idx (bag ids) for the dataloader
# idx = [file[:-4] for file in os.listdir('D:/datasets/Amsterdam/filtered/')] # Complete folder
idx = ['0363100012181176'] # One building

results_summary = intersect(
    idx=idx,
    out_folder='D:/',
    dataset='Amsterdam',
    stepsize=0.1,
    N=10000,
    improvement_threshold=0.0,
    bottom_buffer=2.5,
    top_buffer=2.5,
    smooth=True,
    city_model=city_model,
    city_map=city_map,
    city_outline=city_outline
    )

results_summary

## Convert `.obj` to `.cityjson`

In [ ]:
from pathlib import Path

from cjio import cityjson
from cjio.models import CityObject, Geometry

package_dir = Path(__name__).resolve().parent.parent.parent
schema_dir = package_dir / 'cjio' / 'schemas'/ '1.1.0'
data_dir = package_dir / 'tests' / 'data'

In [ ]:
cm = cityjson.CityJSON()
cm.load_from_j(False)

In [ ]:
# cm.j['transform'] = {
#     "scale": [1.0, 1.0, 1.0],
#     "translate": [0.0, 0.0, 0.0]
#     }

In [ ]:
idx = [file[:-4] for file in os.listdir('D:/datasets/Amsterdam/filtered/')][0:4]

for id in idx:
    # Initiate object
    co = CityObject(
        id=id,
        type='Building'
    )

    # Initiate geometry
    # geom = Geometry(type='Solid', lod=2, transform=cm.j['transform'])
    geom = Geometry(type='Solid', lod=2)

    # Add boundaries
    bdry = [
        [[[0.0, 0.0, 0.0], [10.0, 0.0, 0.0], [10.0, 0.0, 10.0], [0.0, 0.0, 10.0]]],
        [[[10.0, 0.0, 0.0], [10.0, 10.0, 0.0], [10.0, 10.0, 10.0], [10.0, 0.0, 10.0]]],
        [[[10.0, 10.0, 0.0], [0.0, 10.0, 0.0], [0.0, 10.0, 10.0], [10.0, 10.0, 10.0]]],
        [[[0.0, 10.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 10.0], [0.0, 10.0, 10.0]]],
        [[[0.0, 0.0, 0.0], [0.0, 10.0, 0.0], [10.0, 10.0, 0.0], [10.0, 0.0, 0.0]]],
        [[[10.0, 0.0, 10.0], [10.0, 10.0, 10.0], [0.0, 10.0, 10.0], [0.0, 0.0, 10.0]]]
    ]
    geom.boundaries.append(bdry)

    # Add surface information
    geom.surfaces[0] = {'surface_idx': [[0,0], [0,1], [0,2], [0,3]], 'type': 'WallSurface'}
    geom.surfaces[1] = {'surface_idx': [[0,4]], 'type': 'GroundSurface'}
    geom.surfaces[2] = {'surface_idx': [[0,5]], 'type': 'RoofSurface'}

    print(geom.surfaces)
    # Add geometry to City object
    co.geometry.append(geom)
    co.type = 'Building'
    cm.cityobjects[co.id] = co
    cm.add_to_j()

In [ ]:
cm.update_bbox()
print(cm.validate())

In [ ]:
cityjson.save(cm, 'D:/test_create.json')